In [1]:
#import required packages
import pandas as pd
import h2o
from collections import Counter
import numpy as np
from h2o.estimators.gbm import H2OGradientBoostingEstimator


In [2]:
#Prepare the data frame
data = pd.read_csv('DraftResults/Nashville.csv')
data.columns = ['date', 'name1', 'name2', 'draft', 'color', 'splash', 'lands', 'twos', 'rares', 'r1', 'r2', 'r3',
               'exceed', 'failed', 'camp', 'x1', 'x2']
data['color2'] = [x.replace(' ',  '').split(',') for x in data.color]
data['c1'] = [x.replace(' ',  '').split(',')[0] for x in data.color]
data['c2'] = [x.replace(' ',  '').split(',')[1] if len(x.replace(' ',  '').split(',')) == 2 else '' for x in data.color]
data['name'] = data.name1.fillna(data.name2)
data['white'] = [1 if 'White' in colors else 0 for colors in data.color2]
data['blue'] = [1 if 'Blue' in colors else 0 for colors in data.color2]
data['black'] = [1 if 'Black' in colors else 0 for colors in data.color2]
data['red'] = [1 if 'Red' in colors else 0 for colors in data.color2]
data['green'] = [1 if 'Green' in colors else 0 for colors in data.color2]
data['wins'] = [Counter(x)['Win'] for x in zip(data.r1, data.r2, data.r3)]
data['losses'] = [Counter(x)['Lose'] for x in zip(data.r1, data.r2, data.r3)]
data['splash'] = data.splash.fillna(0)
data['splash'] = [0 if x == 0 else 1 for x in data.splash]

In [3]:
data.describe()

/Users/chris/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,draft,splash,lands,twos,rares,x1,x2,white,blue,black,red,green,wins,losses
count,365.000000,365.000000,365.000000,365.000000,365.000000,0.0,0.0,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000
mean,19.241096,0.205479,16.493151,4.876712,1.879452,NaN,NaN,0.402740,0.353425,0.443836,0.367123,0.397260,1.435616,1.421918
std,12.758189,0.404606,0.557745,1.931130,1.223281,NaN,NaN,0.491122,0.478689,0.497518,0.482682,0.490002,0.895251,0.827124
min,1.000000,0.000000,15.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,0.000000,16.000000,4.000000,1.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,19.000000,0.000000,17.000000,5.000000,2.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,30.000000,0.000000,17.000000,6.000000,3.000000,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000
max,42.000000,1.000000,18.000000,11.000000,6.000000,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,3.000000


In [4]:
#Find colorcombination win Percentages
data1 = data.groupby('color').agg({'wins':'sum', 'losses':'sum'})
data1['perc'] = data1.wins/(data1.losses+data1.wins)
data1[['wins', 'losses', 'perc']].sort_values('perc', 0, False)

,wins,losses,perc
color,,,
Red,11,4,0.733333
Green,2,1,0.666667
"White, Blue",53,34,0.609195
"White, Red",57,42,0.575758
"White, Black",71,63,0.529851
"Blue, Red",41,38,0.518987
"Red, Green",49,47,0.510417
Black,3,3,0.500000
Blue,1,1,0.500000


In [18]:
print(data.groupby('draft').agg({'white':'sum', 'blue':'sum', 'black':'sum', 'red':'sum', 'green':'sum'}).mean())
data.groupby('draft').agg({'white':'sum', 'blue':'sum', 'black':'sum', 'red':'sum', 'green':'sum'})


green    3.452381
red      3.190476
black    3.857143
white    3.500000
blue     3.071429
dtype: float64


,green,red,black,white,blue
draft,,,,,
1,7,10,10,8,10
2,8,5,9,8,7
3,6,6,8,6,6
4,6,4,4,2,4
5,5,1,3,6,3
6,2,2,3,2,2
7,4,4,4,2,2
8,2,4,2,4,3
9,1,3,4,5,3


In [6]:
data.groupby('rares').agg({'wins':'sum', 'losses':'sum'})

,wins,losses
rares,,
0,63,68
1,153,139
2,144,170
3,109,91
4,47,39
5,6,11
6,2,1


In [7]:
data.groupby('wins').rares.mean()

wins
0    1.896552
1    1.781955
2    1.954198
3    1.930233
Name: rares, dtype: float64

In [9]:
#Find colorcombination win Percentages
data1 = data.groupby('lands').agg({'wins':'sum', 'losses':'sum'})
data1['perc'] = data1.wins/(data1.losses+data1.wins)
data1[['wins', 'losses', 'perc']].sort_values('perc', 0, False)

,wins,losses,perc
lands,,,
18,2,1,0.666667
15,16,12,0.571429
16,246,226,0.521186
17,260,280,0.481481


In [10]:
#Find colorcombination win Percentages
data1 = data.groupby('splash').agg({'wins':'sum', 'losses':'sum'})
data1['perc'] = data1.wins/(data1.losses+data1.wins)
data1[['wins', 'losses', 'perc']].sort_values('perc', 0, False)

,wins,losses,perc
splash,,,
0,423,404,0.511487
1,101,115,0.467593


In [11]:
Counter(data[data['wins']==3].sort_values('draft')['color'])

Counter({'Black, Green': 3,
         'Black, Red': 3,
         'Blue, Black': 4,
         'Blue, Green': 3,
         'Blue, Red': 3,
         'Red': 2,
         'Red, Green': 5,
         'White, Black': 5,
         'White, Blue': 6,
         'White, Green': 3,
         'White, Red': 6})

In [12]:
#Find colorcombination win Percentages
data1 = data.groupby('twos').agg({'wins':'sum', 'losses':'sum'})
data1['perc'] = data1.wins/(data1.losses+data1.wins)
data1[['wins', 'losses', 'perc']].sort_values('perc', 0, False)


,wins,losses,perc
twos,,,
11,3,0,1.000000
7,50,32,0.609756
8,25,18,0.581395
10,12,9,0.571429
9,20,17,0.540541
4,127,122,0.510040
6,84,82,0.506024
3,74,81,0.477419
5,95,106,0.472637


In [13]:
data1 = data.groupby(['c1', 'c2']).agg({'wins':'sum', 'losses':'sum'})
data1['perc'] = data1.wins/(data1.losses+data1.wins)
data1[['wins', 'losses', 'perc']]

wins  losses      perc
c1    c2                           
Black           3       3  0.500000
      Green    62      67  0.480620
      Red      46      48  0.489362
Blue            1       1  0.500000
      Black    43      56  0.434343
      Green    45      53  0.459184
      Red      41      38  0.518987
Green           2       1  0.666667
Red            11       4  0.733333
      Green    49      47  0.510417
White           3       8  0.272727
      Black    71      63  0.529851
      Blue     53      34  0.609195
      Green    37      54  0.406593
      Red      57      42  0.575758

In [19]:
#Find colorcombination win Percentages
data1 = data.groupby('name').agg({'wins':'sum', 'losses':'sum'})
data1['perc'] = data1.wins/(data1.losses+data1.wins)
data1[['wins', 'losses', 'perc']].sort_values('perc', 0, False)

,wins,losses,perc
name,,,
Eric Froehlich,3,0,1.000000
Ben Weitz,2,0,1.000000
Abe Stein,13,2,0.866667
Sam Sherman,3,1,0.750000
Stephen neal,13,5,0.722222
Pat Cox,2,1,0.666667
Pascal Maynard,16,8,0.666667
Patrick reynolds,4,2,0.666667
Siggy,10,5,0.666667
